In [1]:
import os
import glob
import shutil
import pickle
import subprocess
import cv2
from path import Path
from IPython.display import display

RAW_PATH = '/home/eg4l/Downloads/test3'
UNFILTERED_TMP_PATH = '/home/eg4l/datasets/tmp/unfiltered_tmp_chess'
FILTERED_TMP_PATH = '/home/eg4l/datasets/tmp/filtered_tmp_chess'

WHITE_PATH = '/home/eg4l/datasets/data/white'
BLACK_PATH = '/home/eg4l/datasets/data/black'
EMPTY_PATH = '/home/eg4l/datasets/data/empty'
LABELS_PATH = '/home/eg4l/datasets/labels.pickle'

EMPTY = 0
WHITE = 1
BLACK = 2

WINDOW_NAME = 'is this split correctly? (y/n)'

In [5]:
def correct_rotation():
    for file in glob.glob(RAW_PATH + '/*.*'):

        while True:
            img = cv2.imread(file, cv2.IMREAD_COLOR)
            cv2.imshow('Rotate the picture so that left bottom corner is black', img)

            k = cv2.waitKey(0)

            if k == ord('t'):
                img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
                cv2.imwrite(file, img)
                print(f"Rotating {file} CCW")
            elif k == ord('r'):
                img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
                cv2.imwrite(file, img)
                print(f"Rotating {file} CW")
            elif k == ord('y'):
                break
            elif k == ord('q'):
                print('quitting')
                cv2.destroyAllWindows()
                return
            else:
                print("What is this?")

    print("Done correcting all images")
    cv2.destroyAllWindows()

correct_rotation()

What is this?
Rotating /home/eg4l/Downloads/test3/2021-02-18-09-09-00-280.jpg CCW
Rotating /home/eg4l/Downloads/test3/2021-02-18-09-09-00-280.jpg CW
Rotating /home/eg4l/Downloads/test3/2021-02-18-09-09-00-280.jpg CCW
Rotating /home/eg4l/Downloads/test3/2021-02-18-09-09-00-280.jpg CW
Rotating /home/eg4l/Downloads/test3/2021-02-18-09-09-00-280.jpg CW
Rotating /home/eg4l/Downloads/test3/2021-02-18-09-09-00-280.jpg CCW
quitting


In [2]:
subprocess.run(['/home/eg4l/CLionProjects/ChessboardDetectionLib/ChessboardDetectionLib', RAW_PATH, UNFILTERED_TMP_PATH])

CompletedProcess(args=['/home/eg4l/CLionProjects/ChessboardDetectionLib/ChessboardDetectionLib', '/home/eg4l/Downloads/test3', '/home/eg4l/datasets/tmp/unfiltered_tmp_chess'], returncode=0)

In [6]:
# cv2.namedWindow(WINDOW_NAME)
# cv2.setWindowProperty(WINDOW_NAME, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)
def filter_splits():
    for file in glob.glob(UNFILTERED_TMP_PATH + '/*/_mozaic.*'):
        mozaic = cv2.imread(file, cv2.IMREAD_COLOR)
        mozaic = cv2.resize(mozaic, (int(mozaic.shape[1]*0.8), int(mozaic.shape[0]*0.8)))

        cv2.imshow(WINDOW_NAME, mozaic)
        k = cv2.waitKey(0)

        path = Path(file)
        if k == ord('y'):
            path.parent.move(f'{FILTERED_TMP_PATH}/{path.parent.name}')
            print(f"Moving {file} to {FILTERED_TMP_PATH}")
        elif k == ord('n'):
            shutil.rmtree(path.parent)
            print(f"Removeing {path.parent}")
        elif k == ord('q'):
            cv2.destroyAllWindows()
            return
        else:
            print("What is this?")

    cv2.destroyAllWindows()
    print("Done!")

filter_splits()

In [7]:
def mark_cells(files, labels):
    index = 0
    dh = display('Counter', display_id=True)
    while True:
        if index < 0 or index >= len(files):
            break
        print(f"Marking cell # {index}/{len(files)}")
        # dh.update(f"Marking cell # {index}/{len(files)}")

        file = files[index]
        relative_path = Path(file).relpath(FILTERED_TMP_PATH)
        color = labels.setdefault(relative_path, EMPTY)
        cell = cv2.imread(file, cv2.IMREAD_COLOR)
        cell = cv2.resize(cell, (512, 512))
        if color == EMPTY:
            cv2.circle(cell, (50, 50), 50, (128, 128, 128), 3)
        elif color == WHITE:
            cv2.circle(cell, (50, 50), 50, (255, 255, 255), -1)
        elif color == BLACK:
            cv2.circle(cell, (50, 50), 50, (0, 0, 0), -1)
        elif color == -1:
            cv2.circle(cell, (50, 50), 50, (0, 0, 255), -1)

        cv2.imshow('is this black/white/empty or incorrect (b/w/e or r)', cell)

        k = cv2.waitKey(0)
        if k == ord('['):
            index -= 1
        elif k == ord(']'):
            index += 1
        elif k == ord('b'):
            labels[relative_path] = BLACK
        elif k == ord('w'):
            labels[relative_path] = WHITE
        elif k == ord('e'):
            labels[relative_path] = EMPTY
        elif k == ord('r'):
            labels[relative_path] = -1
        elif k == ord('q'):
            cv2.destroyAllWindows()
            return
        else:
            print("Unknown keypress :<")
        with open(LABELS_PATH, "wb") as labels_file:
            pickle.dump(labels, labels_file)

    cv2.destroyAllWindows()
    print("Done!")

def mark_all_cells():

    if Path(LABELS_PATH).isfile():
        with open(LABELS_PATH, "rb") as file:
            labels = pickle.load(file)
    else:
        labels = {}

    mark_cells(sorted(glob.glob(FILTERED_TMP_PATH + '/*/??.jpg')), labels)

mark_all_cells()

'Counter'

Marking cell # 0/1600
Marking cell # 1/1600
Marking cell # 2/1600
Marking cell # 3/1600
Marking cell # 4/1600
Marking cell # 5/1600
Marking cell # 6/1600
Marking cell # 7/1600
Marking cell # 8/1600
Marking cell # 9/1600
Marking cell # 10/1600
Marking cell # 11/1600
Marking cell # 12/1600
Marking cell # 13/1600
Marking cell # 14/1600
Marking cell # 15/1600
Marking cell # 16/1600
Marking cell # 17/1600
Marking cell # 18/1600
Marking cell # 17/1600
Marking cell # 18/1600
Marking cell # 18/1600
Marking cell # 18/1600
Marking cell # 18/1600
Marking cell # 18/1600
Marking cell # 18/1600
Marking cell # 18/1600
Marking cell # 18/1600
Marking cell # 18/1600
Marking cell # 18/1600
Marking cell # 19/1600
Marking cell # 20/1600
Marking cell # 21/1600
Marking cell # 22/1600
Marking cell # 23/1600
Marking cell # 24/1600
Marking cell # 25/1600
Marking cell # 26/1600
Marking cell # 27/1600
Marking cell # 28/1600
Marking cell # 29/1600
Marking cell # 30/1600
Marking cell # 31/1600
Marking cell # 31/160

In [2]:
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def remove_if_exists(path: Path):
    if path.isfile():
        path.remove()

def split_dataset_based_on_labels(labels):

    ensure_dir(EMPTY_PATH)
    ensure_dir(WHITE_PATH)
    ensure_dir(BLACK_PATH)

    for path, color in labels.items():
        absolute_path = Path(f"{FILTERED_TMP_PATH}/{path}")
        target_filename = f"{absolute_path.parent.name}${absolute_path.name}"

        empty_path = Path(f"{EMPTY_PATH}/{target_filename}")
        white_path = Path(f"{WHITE_PATH}/{target_filename}")
        black_path = Path(f"{BLACK_PATH}/{target_filename}")

        if color == EMPTY:
            absolute_path.copy2(empty_path)
            remove_if_exists(white_path)
            remove_if_exists(black_path)
        elif color == WHITE:
            absolute_path.copy2(white_path)
            remove_if_exists(empty_path)
            remove_if_exists(black_path)
        elif color == BLACK:
            absolute_path.copy2(black_path)
            remove_if_exists(empty_path)
            remove_if_exists(white_path)
        else:
            remove_if_exists(empty_path)
            remove_if_exists(white_path)
            remove_if_exists(black_path)

with open(LABELS_PATH, "rb") as file:
    labels = pickle.load(file)
    split_dataset_based_on_labels(labels)

In [14]:
shutil.rmtree(FILTERED_TMP_PATH)

In [28]:
Path(LABELS_PATH).remove()

Path('/home/eg4l/datasets/labels.pickle')

In [ ]:
# Semi reset opencv windows
cv2.destroyAllWindows()

Hello world!!!
